In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np
import csv
import pickle
import matplotlib.pyplot as plt

In [ ]:
with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\INIT_D", "rb") as outfile:
    init_w = pickle.load(outfile)
with open(r"C:\Users\admin\Desktop\Prediction-Simulation-Data\WEIGHT_BGRAPH", "rb") as outfile:
    bi_graph_w = pickle.load(outfile)

In [ ]:
w = init_w + init_w.T
selected_OD = np.sum(w > 0, axis=0) != 1
w = w[selected_OD, :]
w = w[:, selected_OD]

In [ ]:
print(np.sum(w > 0, axis = 0))
print(np.mean(np.sum(w > 0, axis = 0)))

In [ ]:
temp = w - np.diag(np.diag(w))
np.mean(np.sum(temp, axis=0))

In [ ]:
d = np.sum(w, axis=0)
l = np.diag(d) - w
lam, u = np.linalg.eig(l)

In [ ]:
plt.plot(np.sort(lam)[:20])
plt.show()

In [ ]:
c = u[:, np.argsort(lam, axis=0)]
for i in range(5):
    plt.plot(c[:, i])
plt.show()

In [ ]:
CLUSTER_NUM = 4
clustering = SpectralClustering(
    n_clusters=CLUSTER_NUM,
    n_components=CLUSTER_NUM * 2,
    affinity='precomputed', 
    random_state=0, 
    assign_labels='kmeans').fit(w)
labels = clustering.labels_
# labels = np.random.randint(0, 3, OD_NUM)
for n in range(CLUSTER_NUM):
    print(n, np.sum(labels == n))
# np.max(labels)

In [ ]:
import csv
OD_NUM = np.sum(selected_OD)
OD_filepath = r"C:\Users\admin\Desktop\Prediction-Simulation-Data\OD_2000.csv"
OD_pairs = np.zeros((OD_NUM, 6))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    id_col = head.index("id")
    x1_col = head.index("originX")
    y1_col = head.index("originY")
    x2_col = head.index("destX")
    y2_col = head.index("destY")
    lam_col = head.index("lambda")
    i = j = 0
    for row in data:
        if selected_OD[i]:
            x1 = int(row[x1_col])
            x2 = int(row[x2_col])
            y1 = int(row[y1_col])
            y2 = int(row[y2_col])
            lam = float(row[lam_col])
            OD_pairs[j, :] = np.array([[j, x1, y1, x2, y2, lam]])
            j += 1
        i += 1

In [ ]:
for n in range(CLUSTER_NUM):
    new_OD_pairs = OD_pairs[labels == n, :]
    np.savetxt(r"C:\Users\admin\Desktop\Error-Test-Data-SC\OD_Cluster_" + str(n) + ".csv", new_OD_pairs, fmt=('%d', '%d', '%d', '%d', '%d', '%.2f'), delimiter=',', header="id, originX, originY, destX, destY, lambda", comments='')

In [ ]:
OD_filepath = r"C:\Users\admin\Desktop\Prediction-Simulation-Data\RESULT_2000.csv"
global_predict_result = np.zeros((OD_NUM, 3))
with open(OD_filepath, mode='r') as file:
    data = csv.reader(file)
    head = next(data)
    pw_col = head.index("Pw")
    lw_col = head.index("Lw")
    ew_col = head.index("ew")
    i = j = 0
    for row in data:
        if selected_OD[i]:
            pw = float(row[pw_col])
            lw = float(row[lw_col])
            ew = float(row[ew_col])
            global_predict_result[j, 0] = pw
            global_predict_result[j, 1] = lw
            global_predict_result[j, 2] = ew
            j += 1
        i += 1

In [ ]:
predict_result = np.zeros((OD_NUM, 3))
j = 0
for n in range(CLUSTER_NUM):
    with open(r"C:\Users\admin\Desktop\Error-Test-Data-SC\OD_Cluster_Result_" + str(n) + ".csv", mode='r') as file:
        data = csv.reader(file)
        head = next(data)
        pw_col = head.index("Pw")
        lw_col = head.index("Lw")
        ew_col = head.index("ew")
        for row in data:
            pw = float(row[pw_col])
            lw = float(row[lw_col])
            ew = float(row[ew_col])
            predict_result[j, 0] = pw
            predict_result[j, 1] = lw
            predict_result[j, 2] = ew
            j += 1

In [ ]:
index = np.arange(OD_NUM)[labels == 0]
for n in range(1, CLUSTER_NUM):
    index = np.concatenate((index, np.arange(OD_NUM)[labels == n]))

In [ ]:
index

In [ ]:
global_predict_result = global_predict_result[index, :]

In [ ]:
global_predict_result

In [ ]:
predict_result

In [ ]:
rel_error = np.abs(global_predict_result - predict_result) / global_predict_result * 100

In [ ]:
np.mean(rel_error, axis=0)

In [ ]:
np.max(rel_error, axis=0)

In [ ]:
plt.figure(figsize=(12.8, 9.6))
plt.subplot(221)
plt.plot(np.sort(rel_error[:, 0]), label="pw")
plt.plot(np.sort(rel_error[:, 1]), label="lw")
plt.plot(np.sort(rel_error[:, 2]), label="ew")
plt.ylabel("Relative Error/%")
plt.xlabel("OD Index")
plt.title("Number of Clusters = " + str(CLUSTER_NUM))
plt.legend()
plt.grid(True)
plt.subplot(222)
plt.plot(np.sort(rel_error[:, 0])[:1500], label="pw")
plt.plot(np.sort(rel_error[:, 1])[:1500], label="lw")
plt.plot(np.sort(rel_error[:, 2])[:1500], label="ew")
plt.ylabel("Relative Error/%")
plt.xlabel("OD Index")
plt.title("Number of Clusters = " + str(CLUSTER_NUM))
plt.legend()
plt.grid(True)
plt.subplot(223)
plt.plot(np.sort(rel_error[:, 0])[:1800], label="pw")
plt.plot(np.sort(rel_error[:, 1])[:1800], label="lw")
plt.plot(np.sort(rel_error[:, 2])[:1800], label="ew")
plt.ylabel("Relative Error/%")
plt.xlabel("OD Index")
plt.title("Number of Clusters = " + str(CLUSTER_NUM))
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
global_predict_result[np.argmax(rel_error, axis=0), :]

In [ ]:
predict_result[np.argmax(rel_error, axis=0), :]

In [ ]:
divide = 0
plt.figure(figsize=(12.8, 4.8*CLUSTER_NUM))
for n in range(CLUSTER_NUM):
    rel_error = np.abs(global_predict_result[divide:divide + np.sum(labels == n), :] - predict_result[divide:divide + np.sum(labels == n), :]) / global_predict_result[divide:divide + np.sum(labels == n), :] * 100
    plt.subplot(CLUSTER_NUM, 2, 2*n+1)
    plt.plot(np.sort(rel_error[:, 0]), label="pw")
    plt.plot(np.sort(rel_error[:, 1]), label="lw")
    plt.plot(np.sort(rel_error[:, 2]), label="ew")
    plt.ylabel("Relative Error/%")
    plt.xlabel("OD Index")
    plt.title("Number of Clusters = " + str(np.sum(labels == n)))
    plt.legend()
    plt.grid(True)
    plt.subplot(CLUSTER_NUM, 2, 2*n+2)
    plt.plot(np.sort(rel_error[:, 0])[:np.int64(np.sum(labels == n) * 0.75)], label="pw")
    plt.plot(np.sort(rel_error[:, 1])[:np.int64(np.sum(labels == n) * 0.75)], label="lw")
    plt.plot(np.sort(rel_error[:, 2])[:np.int64(np.sum(labels == n) * 0.75)], label="ew")
    plt.xlabel("OD Index")
    plt.title("Number of Clusters = " + str(np.int64(np.sum(labels == n) * 0.75)))
    plt.legend()
    plt.grid(True)
    print(n, np.sum(labels == n), np.mean(rel_error, axis=0), np.max(rel_error, axis=0))
    divide += np.sum(labels == n)
plt.show()

In [ ]:
rel_error = np.abs(global_predict_result - predict_result) / global_predict_result * 100

In [ ]:
MAX_ERROR_NUM = 500
max_error_idx = np.argsort(rel_error[:, 0])[-MAX_ERROR_NUM:]
# rel_error[max_error_idx, 0]

In [ ]:
cut = np.zeros(OD_NUM)
for n in range(OD_NUM):
    cut[n] = np.sum(w[n, labels != labels[n]])
cut = cut[index]
avg_cut = np.mean(cut)
max_cut = np.zeros(10)
for i in range(10):
    max_cut[i] = np.mean(cut[max_error_idx[50 * i : 50 * (i + 1)]])

In [ ]:
plt.plot(max_cut)
plt.plot(avg_cut*np.ones(10))
print(avg_cut)
plt.show()